In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(20000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (20000 rows)...
Finished inserting 20000 rows into 'products' in 0.35s.


,Tables_in_test_database
0,availability
1,brands
2,categories
3,colors
4,products


Products table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,description,text,YES,,None,
3,brand_id,int,YES,MUL,None,
4,category_id,int,YES,MUL,None,
5,price,float,YES,,None,
6,currency,varchar(10),YES,,None,
7,stock,int,YES,,None,
8,ean,varchar(50),YES,,None,
9,color_id,int,YES,MUL,None,


Brands table


...

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Categories table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Colors table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(100),YES,UNI,None,


So a covering index should be a index that contains all the values needed for the query. And idea is that it should be faster since you dont have to traverse the clustered index to get all the data you need

So for starters i want to test if i select two fields but use only a only a index with a single key that i search for in the db. This should be slower than if it were a covering index even though the where is only on one of the keys

In [2]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1130 rows fetched, 2.83 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size,idx_size,403,const,1130,100.0,None


In [3]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_name ON products(size, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1130 rows fetched, 2.28 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size_and_name,idx_size_and_name,403,const,1130,100.0,Using index


It seems to be faster. I did have to up the records size to 2m in order to see the difference.

I want to try with also using some sql methods in a covering index

In [4]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1 rows fetched, 0.77 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,category_id,category_id,5,const,585,100.0,None


In [5]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_category_id_and_price ON products(category_id, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1 rows fetched, 0.41 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_category_id_and_price,idx_category_id_and_price,5,const,585,100.0,Using index


Here we can see in the extra that it is using the index which just means that it gets all the information it needs from the index and dont have to traverse the clustered index to look at the data. Pretty cool